In [1]:
import boto3

In [8]:
import yaml, os
import zipfile

def get_keys(file_key = 'aws_key.yaml'):
    # Looks for credentials in local file
    if os.path.exists(file_key):
        secrets_dict = yaml.load(open(file_key,'r'), Loader=yaml.FullLoader)
        AWS_ACCESS_KEY = secrets_dict['AWS_ACCESS_KEY']
        AWS_SECRET_KEY = secrets_dict['AWS_SECRET_KEY']        
    # Looks for credentials as environment variables (recommended)
    else:
        AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY']
        AWS_SECRET_KEY = os.environ['AWS_SECRET_KEY']
    return AWS_ACCESS_KEY, AWS_SECRET_KEY

In [18]:
ACCESS_KEY, SECRET_KEY = get_keys(file_key = '../aws_key.yaml')
s3 = boto3.client('s3', region_name = 'us-west-2', 
    aws_access_key_id = ACCESS_KEY,
    aws_secret_access_key = SECRET_KEY)

In [19]:
source_bucket = 'chat-songs'
s3.get_bucket_location(Bucket=source_bucket)

{'ResponseMetadata': {'RequestId': '2MMM6J1FVDAX3TY8',
  'HostId': 'HhaU0L6AhhuQQ7EvHddp8r6S4/lSjwNVrPOKALQpP2N1qLnKuADa6GPFbwSCkTVKP3YFYkssMn4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HhaU0L6AhhuQQ7EvHddp8r6S4/lSjwNVrPOKALQpP2N1qLnKuADa6GPFbwSCkTVKP3YFYkssMn4=',
   'x-amz-request-id': '2MMM6J1FVDAX3TY8',
   'date': 'Sun, 28 May 2023 05:11:01 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'LocationConstraint': 'us-east-2'}

In [25]:
s3.download_file(Bucket='chat-songs',Key='billie_jean.wav',Filename='./song')

In [26]:
polly = boto3.client("polly", region_name = 'us-west-2', 
    aws_access_key_id = ACCESS_KEY,
    aws_secret_access_key = SECRET_KEY)

In [140]:
from botocore.exceptions import BotoCoreError, ClientError

try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text="Hello world!", OutputFormat="mp3",
                                        VoiceId="Ivy")
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print(error)
    sys.exit(-1)

In [124]:
from contextlib import closing

with closing(response["AudioStream"]) as stream:
    with closing(response["AudioStream"]) as stream:
        output = "./speech.mp3"

        try:
            # Open a file for writing the output as a binary stream
            with open(output, "wb") as file:
                file.write(stream.read())
        except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
            sys.exit(-1)

In [90]:
description = "They sit on a fallen tree near the edge of the town, looking \
out at the ruined landscape before them. The sun is setting in the distance, casting \
an orange glow across the sky and the destroyed buildings."

description = "They are walking down the empty town streets. \
    The clouds are grey, and it looks like it's about to rain. They walk past a few \
    fishermen selling their wares, a couple of shopkeepers who seem unusually nervous \
    and a police officer who has a very agitated demeanor."
thomas = "A man with weathered, lined face from years of working on the \
docks. He\'s tall and lanky and wears a fisherman's jacket and rubber \
boots."

evelyn = "Woman of striking appearance, with piercing blue eyes \
and curly black hair. She's lean and athletic, wearing hiking boots and cargo pants."

prompt = description.lower().replace('evelyn',evelyn).replace('thomas',thomas)

prompt = f'{evelyn} {thomas} {description}'

In [93]:
prompt

"Woman of striking appearance, with piercing blue eyes and curly black hair. She's lean and athletic, wearing hiking boots and cargo pants. A man with weathered, lined face from years of working on the docks. He's tall and lanky and wears a fisherman's jacket and rubber boots. They are walking down the empty town streets.     The clouds are grey, and it looks like it's about to rain. They walk past a few     fishermen selling their wares, a couple of shopkeepers who seem unusually nervous     and a police officer who has a very agitated demeanor. Animation style"

In [96]:
negative_prompt = "ugly, poorly drawn face, out of frame, extra limbs, disfigured, deformed, ((face out of frame)), bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed"
negative_prompt = [n.strip() for n in negative_prompt.split(',')]
negative_prompt

['ugly',
 'poorly drawn face',
 'out of frame',
 'extra limbs',
 'disfigured',
 'deformed',
 '((face out of frame))',
 'bad anatomy',
 'watermark',
 'signature',
 'cut off',
 'low contrast',
 'underexposed',
 'overexposed']

In [110]:
path = '../txt2img/lovecraft.yaml'
path = '../txt2img/donaldson.yaml'
params = {
    'defaults':
        {
      "styles": [],
      "seed": 10,
      "subseed": 10,
      "restore_faces": True,
      "cfg_scale": 13,
      "seed_resize_from_h": -1,
      "seed_resize_from_w": -1,
      "sampler_name": "Euler a",
      "batch_size": 1,
      "n_iter": 1,
      "steps": 50,
      "width": 1280,
      "height": 720,
    },
    'style': 'kid\'s cartoon',
    'negative_prompt': negative_prompt
}

yaml.dump(params, open(path,'w'))
params

{'defaults': {'styles': [],
  'seed': 10,
  'subseed': 10,
  'restore_faces': True,
  'cfg_scale': 13,
  'seed_resize_from_h': -1,
  'seed_resize_from_w': -1,
  'sampler_name': 'Euler a',
  'batch_size': 1,
  'n_iter': 1,
  'steps': 50,
  'width': 1280,
  'height': 720},
 'style': "kid's cartoon",
 'negative_prompt': ['ugly',
  'poorly drawn face',
  'out of frame',
  'extra limbs',
  'disfigured',
  'deformed',
  '((face out of frame))',
  'bad anatomy',
  'watermark',
  'signature',
  'cut off',
  'low contrast',
  'underexposed',
  'overexposed']}

In [ ]:
def makedirs(paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)


In [139]:
child_threshold = 14
old_threshold = 50

import requests

import io
import base64
from PIL import Image, PngImagePlugin
from io import BytesIO


def prepare_prompt(scene, characters_data):
    prompt = scene['SCENE_IMAGE'].lower().strip()
    for character in characters_data:
        char_descr = character['character_appearance']
        gender = character['character_gender'].lower().strip()
        char_pronoun = 'he' if gender == 'male' else 'she'

        char_first_name = character['character_name'].split()[0].lower().strip()
        char_name = character['character_name'].lower().strip()
        char_descr = char_descr.lower().strip().replace(char_name,char_pronoun).replace(char_first_name,char_pronoun)

        prompt = prompt.replace(char_name,char_descr).replace(char_first_name,char_descr)
    return prompt

path = '../data/stories/donaldson0/'
prompt_data = '../txt2img/donaldson.yaml'
path = '../data/stories/lovecraft0/'
prompt_data = '../txt2img/lovecraft.yaml'
section = 0


characters_path  = os.path.join(path, '01_characters.yaml')
scene_path = os.path.join(path, '04_1_section_plot.yaml')
characters_data = yaml.load(open(characters_path,'r'), Loader=yaml.FullLoader)
scene_data = yaml.load(open(scene_path,'r'), Loader=yaml.FullLoader)
data = yaml.load(open(prompt_data,'r'), Loader=yaml.FullLoader)
params = data['defaults']

params['negative_prompt'] = ', '.join(data['negative_prompt']) 


path_images = os.path.join(path,'images')
makedirs([path_images])

for i, scene in enumerate(scene_data):
    path_image = os.path.join(path_images,f'img_{section}_{i}.jpeg')
    prompt = prepare_prompt(scene, characters_data)
    params['prompt'] = f"{prompt}, {data['style']}"
    print(params['prompt'])
    if not os.path.exists(path_image):
        r = requests.post('http://192.168.0.13:7860/sdapi/v1/txt2img', json = params)
        Variable= Image.open(io.BytesIO(base64.b64decode(r.json()['images'][0])))
        Variable.save(path_image, format='jpeg')

the scene shows she has a striking appearance, with piercing blue eyes and curly black hair. she's lean and athletic, and usually wears practical clothing, like hiking boots and cargo pants. and he has a weathered, lined face from years of working on the docks. he's tall and lanky, and usually wears a fisherman's jacket and rubber boots. walking down the empty town streets. the clouds are grey, and it looks like it's about to rain. they walk past a few fishermen selling their wares, a couple of shopkeepers who seem unusually nervous and a police officer who has a very agitated demeanor., photorealistic
the scene shows he has a weathered, lined face from years of working on the docks. he's tall and lanky, and usually wears a fisherman's jacket and rubber boots. and she has a striking appearance, with piercing blue eyes and curly black hair. she's lean and athletic, and usually wears practical clothing, like hiking boots and cargo pants. walking quickly along the sidewalk. the sky is dar

In [141]:
yaml.load(open('../../endeavor/video_editor/data/video_scripts/koala_pics.yaml','r'), Loader=yaml.FullLoader)

{'bg_audio': '../data/bg_sounds/American Idle - RKVC.mp3',
 'folder_content': '../data/stock_content/',
 'folder_output': '../data/finished_videos/',
 'name': 'koala_pics.mp4',
 'params': {'resolution': (1280, 720), 'text_fraction': 0.33},
 'snips': [{'content': 'koala_1__koala-bear-on-grey-wood-trunk-on-daytime-162339.jpeg',
   'stacking': 'left_text',
   'text': 'This is a video about a koala'},
  {'content': 'sydney_3__sydney-opera-house-sydney-354375.jpg',
   'stacking': 'right_text',
   'text': 'She was born in Sydney'},
  {'content': 'amalfi_2__white-boat-passing-stone-formations-on-ocean-3009067.jpeg',
   'stacking': 'left_text',
   'text': 'She lived a life in Italy'},
  {'content': 'london_4__black-vehicle-at-the-road-1647120.jpeg',
   'stacking': 'bottom_text',
   'text': 'Then moved to London'},
  {'content': 'chicks_1__bird-chicks-baby-chicken-young-bird-349026.jpg',
   'stacking': 'left_text',
   'text': 'There she met a chicken'},
  {'content': 'marriage_0__heart-wedding-

In [111]:
{
    'content': 'sydney_3__sydney-opera-house-sydney-354375.jpg',
    'stacking': 'right_text',
    'text': 'She was born in Sydney',
    'audio': 
    'video_post_sst_buffer': 2
},

In [177]:
d = dict(
    author_name = 'H. P. Lovecraft',
    author_style = 'atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft\'s prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.',
    data_storage = './data/stories',
    seed = 1,
    bg_audio = 'Voices - Patrick Patrikios.mp3',
    story_summary = 'In a secluded rural town, strange occurrences begin to unfold after a meteor crashes nearby. As the townspeople investigate, they discover that the meteor is not what it seems – it is a vessel containing an ancient extraterrestrial entity with unimaginable powers. The entity slowly infiltrates the minds of the residents, causing madness and transforming them into grotesque, tentacled abominations. A group of survivors must unravel the cosmic secrets behind the entity\'s arrival and find a way to stop its influence before it consumes the entire town and spreads its alien malevolence beyond.',
    story_key = 'lovecraft2',
)

yaml.dump(d,open('lovecraft2.yaml','w'))

In [168]:
import requests
from tqdm import tqdm
nr_sections = 5
story_ref = 'donaldson0'
story_ref = 'lovecraft0'
all_vids = []
for section_id in range(nr_sections):
    ls_screens = yaml.load(open(f'../data/stories/{story_ref}/05_0{section_id}_aud_img_scripts.yaml','r'),Loader = yaml.FullLoader)
    all_screens = []
    for screen in tqdm(ls_screens, desc = f'{story_ref}:::Section {section_id}'):
        full_path = screen['txt2img_image'].replace('./data/','../../../md-similacra/data/')
        image_path = '/'.join(full_path.split('/')[:-1])
        image_name = full_path.split('/')[-1]
        data = {
          "user_id": "user1234",
          "file_name": image_name,
          "input_path": image_path,
          "output_path": "../data/video_summary/assets",
          "text": "Some non-empty text",
          "video_effects": "img",
          "video_fade_in": "0.0",
          "video_fade_out": "0.0",
          "video_post_stt_buffer": "0.5",
          "resolution": [
            1280,
            720
          ],
          "vox": screen['path_audio'].replace('./data/','../../../md-similacra/data/'),
          "text_position": "no_text"
        }

        r = requests.post('http://localhost:8000/byo_vox_video/', json = data)
        all_screens.append(r.json())

    """    
    data = {
      "user_id": "user1234",
      "files_list": [s['output_file'] for s in all_screens],
      "video_summary_name": f"{story_ref}.mp4",
      "input_path": "../data/video_summary/assets",
      "output_path": "../data/video_summary/summaries",
      "bg_audio_file": "../data/bg_sounds/Organic Guitar House - Dyalla.mp3",
      "bg_audio_volume_frac": "0.2",
      "bg_audio_start_sec": "0"
    }

    r = requests.post('http://localhost:8000/list_files_summary_video/', json = data)
    all_vids.append(r.json())
    """
    all_vids.append(all_screens)

lovecraft0:::Section 4: 100%|██████████| 8/8 [00:30<00:00,  3.80s/it]


In [169]:
len(all_vids)

5

In [171]:
import itertools

def flatten_list(ls):
    return list(itertools.chain.from_iterable(ls))


if story_ref == 'lovecraft0':
    bg_audio = 'Voices - Patrick Patrikios.mp3'
elif story_ref == 'donaldson0':
    bg_audio = 'Organic Guitar House - Dyalla.mp3'
data = {
  "user_id": "user1234",
  "files_list": [f['output_file'] for f in flatten_list(all_vids)],
  "video_summary_name": f"{story_ref}.mp4",
  "input_path": "../data/video_summary/assets",
  "output_path": "../data/video_summary/summaries",
  "bg_audio_file": f"../data/bg_sounds/{bg_audio}",
  "bg_audio_volume_frac": "0.2",
  "bg_audio_start_sec": "0"
}

r = requests.post('http://localhost:8000/list_files_summary_video/', json = data)

In [157]:
r.json()

{'user_id': 'user1234',
 'files_list': ['img_00_00_e2dbca24b6bed126dabb86a605cad2cf.mp4',
  'img_00_00_5cef216d8f1ad2608cb8899c7277e596.mp4',
  'img_00_00_a34b91bdb8f8b0b4f5e674a578e0beb6.mp4',
  'img_00_00_de6c13e472703f99373231c34bc50fd7.mp4',
  'img_00_00_3992de80bff1ad5c43477d6e3cc35810.mp4',
  'img_00_01_88115c567e8ab78da532eaec39071579.mp4',
  'img_00_01_d7e8242d1f86b3e619ac232ef4b2d3e8.mp4',
  'img_00_01_50ba6b74644657c2f735e9f338e0d5a2.mp4',
  'img_00_01_53edc49d92687d6160334edd3ffaadef.mp4',
  'img_00_01_1485926b0e27c2479910389c6b84c2be.mp4',
  'img_00_01_9695b5e76af7010e0d7b739dc819b3c3.mp4',
  'img_00_01_1d0a6c0b2818b24c0ac8f4f9516444d8.mp4',
  'img_00_01_e14f744723e288d63488798a4e98b6fc.mp4',
  'img_00_01_fad700b75c6c4407f404c03e865da380.mp4'],
 'video_summary_name': 'my_video.mp4',
 'input_path': '../data/video_summary/assets',
 'output_path': '../data/video_summary/summaries',
 'bg_audio_file': '../data/bg_sounds/Organic Guitar House - Dyalla.mp3',
 'bg_audio_volume_frac'